In [2]:
from collections import defaultdict

In [40]:
tickets = [["ICN", "A"], ["A", "B"], ["A", "C"], ["C", "A"], ["B", "D"]]

In [52]:
graph = defaultdict(list)
visit = set()

In [53]:
for i in tickets:
    a, b = i
    graph[a].append(b)
    visit.add(a)
    visit.add(b)
    
for i in graph:
    graph[i].sort()

In [54]:
tmp = {}
for i in visit:
    tmp[i] = False
visit = tmp

In [36]:
visit

{'D': False, 'B': False, 'A': False, 'ICN': False, 'C': False}

## Problems
---
일반적으로 graph를 초기화 시킨 뒤, 목적지를 오름차순으로 나열하여 진행하면 수월하게 풀릴 것으로 생각.
하지만 `[["ICN", "A"], ["ICN", "B"], ['B', 'ICN']]` 처럼, `'ICN' -> 'A'`를 진행하게 됐을 때, 돌아오는편이 없기에 모든 비행권을 사용하지 못하고, 끝나게 되는 문제가 발생

## Solution
---
* 항공권을 사용할 때 `'A' -> 'B'`라고 가정한다면, `'B'`에서부터 출발하는 항공권이 있는지 확인 $\rightarrow$ 두가지 Case를 생각할 수 있음
    * 첫번쨰. 아예 `'B'`가 `tickets`에 __존재하지 않을 때__
    * 두번째, `'B'`가 `tickets`에 존재하지만, __그 `list`가 비어있을 때__ (`B`로부터 출발하는 항공권을 __모두 사용한 상태__)
  
이 조건들을 고려해서 코드 작성을 생각해보자..

- tour : Queue처럼 사용 중 \
(1).[ ] -> 'ICN' => tour = ['ICN'] \
(2).tour.pop(0) = 'ICN' $\rightarrow$ `not 'ICN' in graph.keys() or len(graph['ICN']) == 0` : 'ICN'이 그래프에 없는 애거나, graph['ICN']의 길이가 0이면 돌아올 표가 없는 것 \
(3).`city = graph['ICN'].pop(0)` : `graph['ICN']`에 연결된 첫번째 도시 뽑기 \
    $\rightarrow$ `not city in graph.keys() or len(graph[city]) == 0` : 해당 도시에서 돌아올 표가 있는지 확인 \
    $\rightarrow$ 없다면, `graph['ICN'].append(city)`로 다시 `graph['ICN']`의 맨 뒤로 보내주기 \
    $\rightarrow$ 있다면, `tour.append(city)`
    

In [79]:
answer = []

tour = ['ICN']

while tour:
    pri = tour.pop(0) # 1. pri = ICN ,tour = [] / 2. pri = 'B', tour = [] / 3. pri = 'ICN' , tour = []
    
    if len(graph[pri]) > 0:
        city = graph[pri].pop(0) # 1. city = graph['ICN'].pop(0) = 'A' / 2. city = graph['B'].pop(0) = 'ICN' / 3. city = graph['ICN'].pop(0) = 'A'
        while not city in graph.keys() or len(graph[city]) == 0: # 1. not 'A' in graph.keys() or len(graph['A']) == 0 = True  / 2. not 'B' in graph.keys() or len(graph['B']) == 0  = False / 3. not 'ICN' in graph.keys() or len(graph['ICN']) == 0 = False / 4. not 'A' in graph.keys() or len(graph['A']) == 0 = True
            if len(graph[pri]) > 0:
                graph[pri].append(city) # 1. graph['ICN'].append('A') / 4. graph['ICN'].append('A')
                city = graph[pri].pop(0) # 1. city = graph['ICN'].pop(0) = 'B' / 4. city = graph['ICN'].pop(0) = 'A'  => **무한반복**
            else:
                tour.append(city)
                break
        else: # 2. city = 'B' / 3. city = 'ICN'
            tour.append(city) # 2. tour = ['B'] / 3. tour = ['ICN']
    answer.append(pri) # 1. answer = ['ICN']  / 2. answer = ['ICN', 'B']
    print(pri, end=' ')

ICN A B D 

In [80]:
answer

['ICN', 'A', 'B', 'D']

In [81]:
graph

defaultdict(list, {'ICN': [], 'A': ['C'], 'C': ['A'], 'B': [], 'D': []})

## Solution (Not my Idea)
---
dfs 기반, graph는 똑같이 작성하되, 아래와 같이 진행
아이디어는 되게 비슷한 거 같은데, 맨 안쪽부터 쌓아 올린다는 점, 그 후에 거꾸로 반환하여, 전체를 탐색한다는게 좀 달랐다.
새롭다..

In [55]:
answer = []
def dfs(s, graph):
    while graph[s]:
        print(f"graph[{s}] : ", graph[s])
        dfs(graph[s].pop(0), graph)
    
    if not graph[s]:
        print("s is ", s)
        answer.append(s)
        return

In [56]:
dfs('ICN', graph)

graph[ICN] :  ['A']
graph[A] :  ['B', 'C']
graph[B] :  ['D']
s is  D
s is  B
graph[A] :  ['C']
graph[C] :  ['A']
s is  A
s is  C
s is  A
s is  ICN


In [25]:
answer[::-1]

['ICN', 'A', 'C', 'A', 'B', 'D']

1. ICN -> A -> B -> D => D is empty => answer.append(D) return => B is empty => answer.append(B) return => A is not empty 
2. ICN -> A -> C -> A ->